In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
import random
import keras_tuner as kt

tfk = tf.keras
tfkl = tf.keras.layers

In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
# Folder in which we save the dataset
dataset_dir = 'splitDataset'

In [ ]:
# We create instances of ImageDataGenerator for preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Generate batches of tensor image data with data augmentation
aug_train_data_gen = ImageDataGenerator(rescale=1/255.,
                                  horizontal_flip=True,
                                  rotation_range=90,
                                  vertical_flip=True,
                                  height_shift_range=0.3,
                                  width_shift_range=0.3,
                                  brightness_range=[0.7,1.3])

# Normalize data in order to have values between 0 and 1
noaug_val_data_gen = ImageDataGenerator(rescale=1/255.)

# Generates the dataset for training 
aug_train_gen = aug_train_data_gen.flow_from_directory(directory=dataset_dir + '/train',
                                                           target_size=(256,256),
                                                           color_mode='rgb',
                                                           classes=None,
                                                           class_mode='categorical',
                                                           batch_size=16,
                                                           shuffle=True,
                                                           seed=seed
                                                           )
# Generates the dataset for validation 
noaug_val_gen = noaug_val_data_gen.flow_from_directory(directory=dataset_dir+'/validation',
                                                           target_size=(256,256),
                                                           color_mode='rgb',
                                                           classes=None,
                                                           class_mode='categorical',
                                                           batch_size=16,
                                                           shuffle=True,
                                                           seed=seed)

In [ ]:
# Set input shape
input_shape = (256, 256, 3)
# Set the number of epochs for training
epochs = 200

In [ ]:
# (Conv + ReLU + Batch Normalization + MaxPool) x 6 + FC x 3
def build_model(input_shape):

    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    conv1 = tfkl.Conv2D(
        filters=16,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(input_layer)
    batch1 = tfkl.BatchNormalization()(conv1)
    pool1 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(batch1)

    conv2 = tfkl.Conv2D(
        filters=32,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool1)
    batch2 = tfkl.BatchNormalization()(conv2)
    pool2 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(batch2)

    conv3 = tfkl.Conv2D(
        filters=32,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool2)
    batch3 = tfkl.BatchNormalization()(conv3)
    pool3 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(batch3)

    conv4 = tfkl.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool3)
    batch4 = tfkl.BatchNormalization()(conv4)
    pool4 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(batch4)

    conv5 = tfkl.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool4)
    batch5 = tfkl.BatchNormalization()(conv5)
    pool5 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(batch5)
    
    conv6 = tfkl.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool5)
    batch6 = tfkl.BatchNormalization()(conv6)
    pool6 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(batch6)
    
    # Build the fully connected part
    flattening_layer = tfkl.Flatten(name='Flatten')(pool6)
    flattening_layer = tfkl.Dropout(0.3, seed=seed)(flattening_layer)
    classifier_layer = tfkl.Dense(units=512, name='Classifier', kernel_initializer=tfk.initializers.GlorotUniform(seed), activation='relu')(flattening_layer)
    classifier_layer = tfkl.Dropout(0.4, seed=seed)(classifier_layer)
    classifier_layer1 = tfkl.Dense(units=512, name='Classifier1', kernel_initializer=tfk.initializers.GlorotUniform(seed), activation='relu')(classifier_layer)
    classifier_layer1 = tfkl.Dropout(0.2, seed=seed)(classifier_layer1)
    classifier_layer2 = tfkl.Dense(units=256, name='Classifier2', kernel_initializer=tfk.initializers.GlorotUniform(seed), activation='relu')(classifier_layer1)
    classifier_layer2 = tfkl.Dropout(0.2, seed=seed)(classifier_layer2)
    output_layer = tfkl.Dense(units=14, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed), name='Output')(classifier_layer2)

    # Istantiate the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics=['accuracy',tfk.metrics.AUC(),tfk.metrics.Precision(),tfk.metrics.Recall()])

    # Return the model
    return model

In [ ]:
# Utility function to create folders and callbacks for training
from datetime import datetime

def create_folders_and_callbacks(model_name):

  exps_dir = os.path.join('data_callbacks_experiments')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%b%d_%H-%M-%S')

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []

  # Creation of model checkpoint
  ckpt_dir = os.path.join(exp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  # Callback to save the model
  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'), 
                                                     save_weights_only=False, # True to save only weights
                                                     save_best_only=False) # True to save only the best epoch 
  callbacks.append(ckpt_callback)

  # Early Stopping
  es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
  callbacks.append(es_callback)
  # Method to reduce learning rate when the monitored metric has stopped improving
  reduce_rl_callback=tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=4, factor=0.4, min_lr=0.00001,verbose=1),
  callbacks.append(reduce_rl_callback)  
  
  return callbacks

In [ ]:
# Instantiate the model
model = build_model(input_shape)
# Show the structure of the model
model.summary()

In [ ]:
# Create folders and callbacks
aug_callbacks = create_folders_and_callbacks(model_name='MarkZuckerberg')

# Train the model
history = model.fit(x = aug_train_gen,epochs = epochs,validation_data = noaug_val_gen,callbacks = aug_callbacks).history

In [ ]:
# Save the model
model.save("experiments/MarkZuckerberg")